In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
#Import the dataset Churn_Modeling.csv file
dataset = pd.read_csv('Churn_Modelling.csv')


In [2]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#X values, get all the rows and get relevant columns (3-12)
X = dataset.iloc[:, 3:13].values

#y value, get all the rows and get the relevant output column (13)
y = dataset.iloc[:, 13].values

# Encoding categorical data

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
#Label encode the countries to be distinct integer values
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
#Label encode male and female (1,0) (0,1)
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
#One hot encoder, transform the column Geography (1,0,0) (0,1,0) (0,0,1) to three columns
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

#Get rid of the first column keep all rows because based on columns 2 and 3 you could deduce
#column 1 without having column 1
X = X[:, 1:]
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
#NOTE THIS CODE, this is easy reusable code that gives in this example 80% to the training set and 20% to the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
#Easy feature scaling reusable code to scale the columns in the X training and X test set
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
#Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

#Build the standard classifier to be used later in the code
def build_classifier():
    from keras.models import Sequential
    from keras.layers import Dense

    # Initialising the ANN
    classifier = Sequential()
    
    # Adding the input layer and the first hidden layer
    
    #Add the first hidden layer to have 6 nodes, let the initial values be close to 0 but 'uniform' 
    #use the rectifier function ('relu') and note that the number of input features is 11
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    
    # Adding the second hidden layer
    #Make a second hidden layer and since this is the second layer you do not need
    #to state the number of input dimension but everything else remains the same as above
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    
    # Adding the output layer
    #Create an output layer that has one output and give it an initial value close to 0
    #and get the activation function to be sigmoid because it is between 0 and 1 probability float value
    #and we want the probability that someone will leave the bank 
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    # Compiling the ANN
    #This is the one I will have to research most
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return classifier

#Build the KerasClassifier with a batch size of 10 and a epoch of 100 times
classifier = KerasClassifier(build_fn = build_classifier(), batch_size = 10, nb_epoch= 100)
#Set the accuracies to be a cross_validation using the classifier built and the X_train and y_train with a 10 fold
#cross validation and use all cpus
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1 )

Using TensorFlow backend.
